In [1]:
import os,sys
from ipyparallel import Client
from collections import defaultdict
import tempfile
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import socket
import stopwatch
from multiprocessing import Pool, Manager
import shutil
import tempfile
from collections import OrderedDict
import pandas as pd
import numpy as np

In [393]:
import shutil

In [21]:
file_dir = "/gpfs_fs/home/lindb/wbp/raw_reads"
proc_files = []
for root, dirs, files in os.walk(file_dir):
    for f in files:
        if f.startswith('WBP'):
            proc_files.append(os.path.join(root,f))
len(proc_files)

735

In [23]:
proc_files[199]

'/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP02_2/WBP02_2_NoIndex_L004_R1_024.fastq'

In [24]:
sorted(proc_files)[0]

'/gpfs_fs/home/lindb/wbp/raw_reads/150bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_001.fastq'

In [25]:
proc_files

['/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_001.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_002.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_003.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_004.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_005.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_006.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_007.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_008.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_009.fastq',
 '/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_010.fastq',
 '/gpfs_fs/home/lind

In [1]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

NameError: name 'Client' is not defined

In [188]:
def format_fastq_tuple(title, seq, qual):
    assert len(seq) == len(qual)
    return "@%s\n%s\n+\n%s\n" % (title, seq, qual)

def get_writers(barcodes, f):
    import tempfile
    from collections import defaultdict
    w = defaultdict()
    file_list = []
    for b, name in list(barcodes.items()):
        file_name = "%s_%s_%s.fastq" % (f, name, b)
        file_list.append(file_name)
        w[b] = open(file_name, "w")
    return w, file_list

def get_barcodes(f):
    from collections import defaultdict
    import os
    bcs = defaultdict()
    bc_lens = set()
    b = os.path.join(os.path.dirname(f), "barcodes.txt")
    h = open(b)
    h.readline()
    for line in h:
        line = line.strip()
        data = line.split("\t")
        if len(data) > 1: #skip blank lines
            if "," in data[0]:
                data[0] = data[0].replace(" ", "").replace(",", "-")
            bc = data[4].upper().replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "")[:-1]
            bc_lens.add(len(bc))
            bcs[bc] = data[1] + "_" + data[0]        
    return bcs, bc_lens    

def check_barcodes(barcodes):
    for i in list(barcodes.keys()):
        for j in list(barcodes.keys()):
            if i != j:
                assert not j.startswith(i)
                
def copy_file(src, dst):
    import shutil
    shutil.copy(src, dst)
                
def demult(f):
    print(f)
    import socket, stopwatch, os
    from Bio.SeqIO.QualityIO import FastqGeneralIterator
    from multiprocessing import Pool
    from collections import defaultdict
    hostname = socket.gethostname()
    timer = stopwatch.Timer()
    out_dir = os.path.dirname(f)
    barcodes, barcode_lens = get_barcodes(f)
    check_barcodes(barcodes)
    writers, writer_list = get_writers(barcodes, f)
    count = 0
    found = defaultdict(int)
    bad_barcode = 0
    n_start = 0
    for title, seq, qual in FastqGeneralIterator(open(f)):

        if seq.startswith("N"):
            seq = seq[1:]
            qual = qual[1:]
            n_start += 1
        
        has_barcode = False    
        
        for l in barcode_lens: 
            bc = seq[0:l]
            if bc in barcodes:
                found[bc] += 1
                has_barcode = True
                w = writers[bc]
                w.write(format_fastq_tuple(title, seq[l:], qual[l:]))
                break
            
        count += 1
            
        if count % 100000 == 0:
            print(hostname, f, count)
        
        if not has_barcode:
            bad_barcode += 1
    
    for k, v in writers.items():
        v.close()
            
    return f, count, bad_barcode, found, writer_list

dview['format_fastq_tuple'] = format_fastq_tuple
dview['demult'] = demult
dview['check_barcodes'] = check_barcodes
dview['get_writers'] = get_writers
dview['get_barcodes'] = get_barcodes
dview['copy_file'] = copy_file


In [28]:
demult_jobs = []
for f in proc_files:
    demult_jobs.append(lview.apply_async(demult, f))

In [158]:
np.sum([j.ready() for j in demult_jobs])

735

In [159]:
len(demult_jobs)

735

In [51]:
proc_files[0]

'/gpfs_fs/home/lindb/wbp/raw_reads/50bp_reads/Sample_WBP01_1/WBP01_1_NoIndex_L001_R1_001.fastq'

In [160]:
import pickle

In [161]:
demult_results = [j.r for j in demult_jobs if j.ready()]

In [162]:
pickle.dump(demult_results, open("/home/lindb/wbp/raw_reads/demult_results.pkl", "w"))

In [163]:
demult_results = pickle.load(open("/home/lindb/wbp/raw_reads/demult_results.pkl", "r"))

In [164]:
samp_dict = {}
for res in demult_results:
    files = res[-1]
    for f in files:
        d = f.split("_")
        sample = d[-2]
        if not sample in samp_dict:
            samp_dict[sample] = []
        samp_dict[sample].append(f)

In [180]:
len(samp_dict.keys())

252

In [183]:
sorted(samp_dict.keys())

['10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '20',
 '200',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '207',
 '208',
 '209',
 '21',
 '210',
 '211',
 '212',
 '213',
 '214',
 '215',
 '216',
 '21

# concatenate demult to individual files

In [165]:
file_dir = "/gpfs_fs/home/lindb/wbp/raw_reads"
file_dir

'/gpfs_fs/home/lindb/wbp/raw_reads'

In [184]:
good_dict = {}
for s in samp_dict:
    if s == 'NA':
        pass
    elif s == 'nan':
        pass
    elif s == 'b11':
        pass
    elif s == 'a3':
        pass
    elif s == 'a4':
        pass
    elif s == 'a6':
        pass
    elif s == 'a7':
        pass
    elif s == 'a9':
        pass    
    elif s == 'b11':
        pass
    elif s == 'b9':
        pass    
    else:
        good_dict[s] = samp_dict[s]

In [185]:
len(good_dict.keys())

244

In [186]:
for samp in good_dict.keys():
    for f in good_dict[samp]:
        if not os.path.exists(f):
            print f

In [189]:
with dview.sync_imports():
    import os
    from collections import defaultdict
    from Bio.SeqIO.QualityIO import FastqGeneralIterator
    import socket
    from collections import OrderedDict

importing os on engine(s)
importing defaultdict from collections on engine(s)
importing FastqGeneralIterator from Bio.SeqIO.QualityIO on engine(s)
importing socket on engine(s)
importing OrderedDict from collections on engine(s)


In [190]:
hosts = dview.apply(socket.gethostname)

In [191]:
host_dict = {}
for i, host in enumerate(hosts.get()):
    if not host in host_dict:
        host_dict[host] = []
    host_dict[host].append(i)
host_dict

{'godel10': [2, 3, 60, 79, 96],
 'godel14': [29, 32, 49, 59, 83, 93],
 'godel15': [39, 40, 45, 67, 86, 104, 111],
 'godel16': [31, 35, 63, 82, 100, 123, 144, 145, 146],
 'godel19': [6, 28, 47, 70, 89, 117, 118, 133],
 'godel2': [12, 15, 51, 68, 88, 105, 114],
 'godel20': [18, 23, 54, 78, 97, 112, 125],
 'godel22': [10, 19, 42, 72, 90, 120, 121, 134, 135],
 'godel24': [5, 34, 65, 81, 103, 127, 139, 143, 147],
 'godel26': [4, 30, 48, 73, 91, 110, 137, 138, 142, 148, 149],
 'godel27': [37, 38, 53],
 'godel28': [26, 27, 56, 77, 101, 115, 130],
 'godel29': [20, 21, 50, 69, 98, 113, 126],
 'godel31': [8, 36, 44, 62, 84, 106, 128, 132, 136, 140, 141],
 'godel32': [24, 25, 52, 71, 95, 109, 119],
 'godel34': [7, 11, 55, 74],
 'godel35': [13, 22, 58, 75, 94],
 'godel36': [14, 17, 57, 76, 92, 108, 124],
 'godel5': [0, 1, 43, 61, 85, 107, 122, 129, 131],
 'godel6': [33, 46, 64, 80, 102, 116],
 'godel7': [9, 16, 41, 66, 87, 99]}

In [192]:
single_host = rc.load_balanced_view(targets=[v[0] for k,v in host_dict.items()])

In [193]:
len(single_host)

21

In [383]:
#old don't use
#@lview.remote()
#def old_join_fastq(args):
    joined, file_list = args
    !touch {joined}
    for f in file_list:
        !cat {f} >> {joined}
    return joined

In [384]:
@lview.remote()
def join_fastq(args):
    joined, file_list = args
    for f in file_list:
        for title,seq,qual in FastqGeneralIterator(open(f)):
            with open(joined,'a') as o:
                o.write(format_fastq_tuple(title,seq,qual))
    return joined

In [385]:
#old_jobs = []
#for sample_id, file_list in good_dict.items():
#    if sample_id == '12':
#        new_base_name = sample_id + "TESTcompiled.fastq"
#        dir_name = "/gpfs_fs/home/lindb/wbp/concatenated"
#        joined = os.path.join(dir_name,new_base_name)
#        old_jobs.append(old_join_fastq((joined,file_list)))

In [206]:
len(good_dict.keys())

244

In [207]:
good_dict.keys()

['216',
 '217',
 '214',
 '215',
 '212',
 '213',
 '210',
 '211',
 '218',
 '219',
 '133',
 '91',
 '131',
 '130',
 '137',
 '136',
 '135',
 '134',
 '139',
 '166',
 '24',
 '25',
 '26',
 '92',
 '20',
 '21',
 '22',
 '23',
 '160',
 '28',
 '29',
 '94',
 '221',
 '163',
 '227',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '118',
 '59',
 '58',
 '132',
 '55',
 '54',
 '57',
 '56',
 '51',
 '50',
 '53',
 '52',
 '201',
 '199',
 '198',
 '147',
 '195',
 '194',
 '197',
 '196',
 '191',
 '190',
 '193',
 '192',
 '115',
 '114',
 '117',
 '116',
 '111',
 '110',
 '113',
 '253',
 '82',
 '83',
 '80',
 '81',
 '119',
 '87',
 '165',
 '85',
 '251',
 '207',
 '173',
 '141',
 '226',
 '172',
 '254',
 '255',
 '225',
 '245',
 '244',
 '247',
 '27',
 '241',
 '240',
 '243',
 '242',
 '102',
 '103',
 '100',
 '101',
 '249',
 '248',
 '104',
 '105',
 '39',
 '38',
 '33',
 '32',
 '31',
 '30',
 '37',
 '108',
 '35',
 '34',
 '109',
 '252',
 '151',
 '223',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '17

In [195]:
#send off files to be concatenated
join_jobs = []
for sample_id, file_list in good_dict.items():
    new_base_name = sample_id + "compiled.fastq"
    dir_name = "/gpfs_fs/home/lindb/wbp/concatenated"
    joined = os.path.join(dir_name, new_base_name)
    join_jobs.append(join_fastq((joined, file_list)))

In [196]:
len(join_jobs)

244

In [382]:
#check progress
np.sum([j.ready() for j in join_jobs])

244

In [413]:
#get a list of concatenated files
compdir = '/home/lindb/wbp/concatenated/'
files = os.listdir(compdir)
files = [os.path.join(compdir,f) for f in files if 'compiled' in f]
len(files)

246

In [414]:
#check to make sure nothing is wonky
#if everything is good, no errors should print
for f in files:
    for t,s,q in FastqGeneralIterator(open(f)):
        pass

KeyboardInterrupt: 

In [395]:
@lview.remote()
def do_it(f):
    from Bio.SeqIO.QualityIO import FastqGeneralIterator
    import os,sys
    for t,s,q in FastqGeneralIterator(open(f)):
        pass
    return f

In [396]:
lst = []
for f in files:
    lst.append(do_it(f))

In [397]:
#check progress
np.sum([j.ready() for j in lst])

244

In [398]:
for j in lst:
    if j.ready():
        if not j.r.startswith('/home/'):
            print j.r

In [403]:
#having sample IDs of the same length is handy!
for f in files:
    bname = os.path.basename(f)
    if len(bname) == 17:
        new_name = bname
    if len(bname) == 16:
        new_name = '0' + bname
    if len(bname) == 15:
        new_name = '00' + bname
    if not len(new_name) == 17:
        print new_name
    dst = os.path.join(compdir,new_name)
    shutil.move(f,dst)

In [410]:
#file with greatest reads = 012compiled.fastq
#split into 50bp or 150bp for VelvetOptimiser.pl
f = '/home/lindb/wbp/concatenated/012compiled.fastq'
w50 = open("/%s_50.fastq" % f,  "w")
w150 = open("/%s_150.fastq" % f, "w")
for t, s, q in FastqGeneralIterator(open(f)):
    w = w50
    if len(s) > 50:
        w = w150
    w.write(format_fastq_tuple(t, s, q))
w50.close()
w150.close()

In [412]:
#check one last time
fz = ['/home/lindb/wbp/concatenated/012compiled.fastq_150.fastq','/home/lindb/wbp/concatenated/012compiled.fastq_50.fastq']
for f in fz:
    for t,s,q in FastqGeneralIterator(open(f)):
        pass